In [1]:
import tensorflow as tf

In [2]:
model1 = tf.keras.models.load_model('value_model_350')

2022-06-04 12:20:35.149583: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-04 12:20:35.747307: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14807 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:3f:00.0, compute capability: 7.5


In [3]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 350)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 350)]        0           []                               
                                                                                                  
 bert (Custom>TFBertMainLayer)  {'last_hidden_state  108310272   ['input_ids[0][0]',              
                                ': (None, 350, 768)               'attention_mask[0][0]']         
                                , 'pooler_output':                                                
                                (None, 768)}                                                  

In [4]:
from transformers import BertTokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [10]:
seq_len = 350
def prep_data(text):
    tokens = tokenizer.encode_plus(text, max_length = 350, truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False, return_tensors='tf')
    return {
        'input_ids':tf.cast(tokens['input_ids'], tf.float32),
        'attention_mask':tf.cast(tokens['attention_mask'], tf.float32)
    }

In [11]:
import pandas as pd

test_df = pd.read_csv('Data/test.csv')
train = pd.read_csv('train.csv')

In [12]:
#test_df.drop('Unnamed: 0', inplace=True, axis=1)
test_df['label'] = test_df['label'] + 1

train['label'] = train['label'] + 1
train.head()

,scenario,label
0,[POWER] After accomplishing every task I cross...,2
1,"[CONFORMITY] when i was a young boy, i told on...",0
2,[BENEVOLENCE] getting mad at my mom for not le...,0
3,[BENEVOLENCE] sleeping with my friend's roomat...,0
4,[BENEVOLENCE] laughing about a boy's disgrace,0


In [13]:
import numpy as np

In [ ]:
total=0
for i, thing in enumerate(train['scenario']):
    token = prep_data(thing)
    result = np.argmax(model1.predict(token))
    if result == train['label'][i]:
        total += 1

In [ ]:
total_test=0
for i, thing in enumerate(test_df['scenario']):
    token = prep_data(thing)
    result = np.argmax(model1.predict(token))
    if result == test_df['label'][i]:
        total_test += 1

In [15]:
print(total_test/len(train))

0.040161346895826025
